In [1]:
import pandas as pd
from pandas.api.types import is_object_dtype, is_numeric_dtype
import numpy as np
from tqdm import tqdm
import plotly.express as px
import statsmodels.api as sma
from sklearn import preprocessing 
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif, SelectKBest, SequentialFeatureSelector
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier
from sklearn.metrics import f1_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("Breast Cancer/data.csv")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

#Dzielimy dane na uczące i testowe 
X = data.drop(['id','diagnosis'], axis=1)
y = data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.3,
                                                   random_state=42)

# Initialize classifier
gbc = GradientBoostingClassifier(max_depth=5, random_state=42)  # Dla problemu klasyfikacji

In [186]:
# INNE PODEJSCIE POD ACO 
# POMYŚLEĆ O SKALOWANIU
# Krok 1 Ranking parametrów
mutual_info = mutual_info_classif(X_train, y_train)
weights = np.array(mutual_info)
print("Wagi:\n",weights)

Wagi:
 [0.32618747 0.10177276 0.37107807 0.32882171 0.0794367  0.20089121
 0.36897949 0.44488804 0.04502213 0.02629505 0.25094202 0.01141171
 0.24998774 0.31697764 0.01688705 0.04094205 0.12616505 0.10974477
 0.02026062 0.00641461 0.4412525  0.1670385  0.45254054 0.43881309
 0.09618387 0.21113197 0.31459994 0.44550612 0.08302258 0.06048958]


In [187]:
# KROK 2
# DEFINIUJE PHEROMONE ARRAY (TAKA SAMA WARTOSC)
def generate_pheromone_array(num_features, value):
    pheromone_array = np.full(num_features, value)
    return pheromone_array

In [188]:
pheromone_array = generate_pheromone_array(30, 0.2)
print("Pheromone_array: \n", pheromone_array)

Pheromone_array: 
 [0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2
 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2]


In [ ]:
# Krok 3
# Generacja początkowego ustawienia mrówek
def generate_ant_population(ants_colony_size, num_features):
    ants = np.zeros(shape=(ants_colony_size, num_features), dtype=int)
    for ant in ants:
        random_node_index = np.random.randint(0, num_features)
        ant[random_node_index] = 1
    print("Ants generated!\n")
    return ants

ants = generate_ant_population(30, 30)
print("Ants:\n", ants)

In [180]:
from itertools import accumulate
import operator

# KROK 4 KALKULACJA PRAWDOPODBIENSTWA DLA KANDYDATOW i SELEKCJA NOWEJ CECHY
alpha = 1.0
beta = 1.0

print("Initial Ants Population:\n", ants)

# Implement Roulette Wheel Selection
def roulette_wheel_selection(probabilities, num_candidates):
    cumulative_probabilities = list(accumulate(probabilities, operator.add))
    print(cumulative_probabilities)
    selected_index = np.searchsorted(cumulative_probabilities, np.random.rand()) % num_candidates
    return selected_index

def ants_make_move(ants_population):
    for ant_index in range(len(ants_population)):
        current_ant = ants_population[ant_index]
        
        print("Ant {}:".format(ant_index + 1))
        print(current_ant)
    
        # Calculate the sum of probabilities for normalization
        total_probability = np.sum((pheromone_array[current_ant == 0] ** alpha) * (weights[current_ant == 0] ** beta))
        probabilities_for_each_candidate = np.zeros(len(current_ant))
        
        for j in range(len(current_ant)):
            if current_ant[j] == 0:
                pheromone_level = pheromone_array[j]
                attractiveness = weights[j]
                probability = ((pheromone_level ** alpha) * (attractiveness ** beta)) / total_probability
                probabilities_for_each_candidate[j] = probability
    
        selected_feature_index = roulette_wheel_selection(probabilities_for_each_candidate, len(probabilities_for_each_candidate))
        print("Selected Feature Index:", selected_feature_index)
        print()
        current_ant[selected_feature_index] = 1
        print("Ant selected:\n", current_ant)
        print()

    return ants_population


ants_population_after_move = ants_make_move(ants)

print("New ants population\n:", ants_population_after_move)

Initial Ants Population:
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0

In [183]:
# Krok 5 SPRAWDZ FITNESS MRÓWEK
def calculate_ant_fitness(features):
   feature_mask = features.astype(bool)
   X_train_subset = X_train.loc[:, feature_mask]
   X_test_subset = X_test.loc[:, feature_mask]
   gbc.fit(X_train_subset, y_train)
   preds = gbc.predict(X_test_subset)
   f1_score_subset = round(f1_score(y_test, preds, average='weighted'), 3)
   return f1_score_subset

In [184]:
fitness_scores_ants = [calculate_ant_fitness(features) for features in tqdm(ants_population_after_move, desc='Fitness Score Calculation Progress', colour = "green", leave=True)]
print(fitness_scores_ants)

Fitness Score Calculation Progress: 100%|██████████| 30/30 [00:08<00:00,  3.42it/s]

[0.924, 0.906, 0.959, 0.959, 0.959, 0.959, 0.918, 0.936, 0.866, 0.947, 0.959, 0.93, 0.942, 0.89, 0.965, 0.836, 0.953, 0.901, 0.942, 0.93, 0.941, 0.936, 0.906, 0.906, 0.889, 0.93, 0.929, 0.919, 0.924, 0.93]


In [185]:
# KROK 6 ZAKTUALIZUJ LISTE FEROMONÓW NA PODSTAWIE WYBORÓW MRÓWEK
def update_pheromones(pheromone_array, ants_population, quality_measure, evaporation_rate=0.1):
    pheromone_array *= (1 - evaporation_rate)
    for ant in ants_population:
        for j in range(len(pheromone_array)):
            if ant[j] == 1:
                # Update pheromone levels based on the quality measure
                pheromone_array[j] += quality_measure[j]
    return pheromone_array

updated_pheromones = update_pheromones(pheromone_array, ants_population_after_move, fitness_scores_ants)
print("Updated Pheromone Levels:\n", updated_pheromones)

Updated Pheromone Levels:
 [5.724 1.992 4.016 3.057 1.139 2.098 3.852 2.052 1.912 2.074 4.016 1.11
 5.832 4.63  1.145 1.852 2.086 2.883 0.18  0.18  5.826 2.988 4.71  2.898
 1.958 5.76  3.896 8.451 0.18  0.18 ]


In [ ]:
# KROK 7 WYKONUJEMY KOLEJNE KROKI MRÓWEK

In [3]:
# TEST
# Krok 1 Ranking parametrów
mutual_info = mutual_info_classif(X_train, y_train)


# KROK 2
# DEFINIUJE PHEROMONE ARRAY (TAKA SAMA WARTOSC)
def generate_pheromone_array(num_features, value):
    pheromone_array = np.full(num_features, value)
    return pheromone_array

# Krok 3
# Generacja początkowego ustawienia mrówek
def generate_ant_population(ants_colony_size, num_features):
    ants = np.zeros(shape=(ants_colony_size, num_features), dtype=int)
    for ant in ants:
        random_node_index = np.random.randint(0, num_features)
        ant[random_node_index] = 1
    print("Ants generated!\n")
    return ants

from itertools import accumulate
import operator

# KROK 4 KALKULACJA PRAWDOPODBIENSTWA DLA KANDYDATOW i SELEKCJA NOWEJ CECHY

# Implement Roulette Wheel Selection
def roulette_wheel_selection(probabilities, num_candidates):
    print("Problities:\n", probabilities)
    cumulative_probabilities = list(accumulate(probabilities, operator.add))
    print("Cumulative probabilities: \n", cumulative_probabilities)
    selected_index = np.searchsorted(cumulative_probabilities, np.random.rand()) % num_candidates
    return selected_index

def ants_make_move(ants_population, pheromone_array, weights):
    for ant_index in range(len(ants_population)):
        current_ant = ants_population[ant_index]
        
        print("Ant {}:".format(ant_index + 1))
        print(current_ant)
    
        # Calculate the sum of probabilities for normalization
        total_probability = np.sum((pheromone_array[current_ant == 0] ** alpha) * (weights[current_ant == 0] ** beta))
        probabilities_for_each_candidate = np.zeros(len(current_ant))
        
        for j in range(len(current_ant)):
            if current_ant[j] == 0:
                pheromone_level = pheromone_array[j]
                attractiveness = weights[j]
                probability = ((pheromone_level ** alpha) * (attractiveness ** beta)) / total_probability
                probabilities_for_each_candidate[j] = probability
    
        selected_feature_index = roulette_wheel_selection(probabilities_for_each_candidate, len(probabilities_for_each_candidate))
        print("Selected Feature Index:", selected_feature_index)
        print()
        current_ant[selected_feature_index] = 1
        print("Ant selected:\n", current_ant)
        print()

    return ants_population

# Krok 5 SPRAWDZ FITNESS MRÓWEK
def calculate_ant_fitness(features):
   feature_mask = features.astype(bool)
   X_train_subset = X_train.loc[:, feature_mask]
   X_test_subset = X_test.loc[:, feature_mask]
   gbc.fit(X_train_subset, y_train)
   preds = gbc.predict(X_test_subset)
   f1_score_subset = round(f1_score(y_test, preds, average='weighted'), 3)
   return f1_score_subset

# KROK 6 ZAKTUALIZUJ LISTE FEROMONÓW NA PODSTAWIE WYBORÓW MRÓWEK
def update_pheromones(pheromone_array, ants_population, quality_measure, evaporation_rate=0.1):
    pheromone_array *= (1 - evaporation_rate)
    for ant in ants_population:
        for j in range(len(pheromone_array)):
            if ant[j] == 1:
                # Update pheromone levels based on the quality measure
                pheromone_array[j] += quality_measure[j]
    return pheromone_array

alpha = 1.0
beta = 1.0
ants = generate_ant_population(30, 30)
weights = np.array(mutual_info)
pheromone_array = generate_pheromone_array(30, 0.2)
print("Initial ants population:\n", ants)
print("Wagi:\n",weights)
print("Initial pheromone_array: \n", pheromone_array)

for i in range(5): 
    ants_population_after_move = ants_make_move(ants, pheromone_array=pheromone_array, weights=weights)
    fitness_scores_ants = [calculate_ant_fitness(features) for features in tqdm(ants_population_after_move, desc='Fitness Score Calculation Progress', colour = "green", leave=True)]
    print(max(fitness_scores_ants))
    print(fitness_scores_ants)
    updated_pheromones = update_pheromones(pheromone_array, ants_population_after_move, fitness_scores_ants)
    print("Updated Pheromone Levels:\n", updated_pheromones)
    print("New ants population\n:", ants_population_after_move)

print(np.argmax(fitness_scores_ants))


Ants generated!

Initial ants population:
 [[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 

Fitness Score Calculation Progress: 100%|██████████| 30/30 [00:01<00:00, 16.80it/s]


0.953
[0.871, 0.935, 0.901, 0.948, 0.901, 0.802, 0.84, 0.889, 0.873, 0.895, 0.936, 0.82, 0.895, 0.941, 0.629, 0.924, 0.912, 0.883, 0.912, 0.873, 0.866, 0.936, 0.866, 0.93, 0.935, 0.877, 0.936, 0.889, 0.953, 0.936]
Updated Pheromone Levels:
 [2.793 0.18  1.982 3.024 1.081 2.586 1.86  1.069 3.672 0.18  2.988 0.18
 0.18  2.062 0.18  0.18  0.18  0.18  1.092 1.053 7.974 1.116 3.644 2.97
 2.05  1.057 5.796 2.847 3.039 2.052]
New ants population
: [[0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0

Fitness Score Calculation Progress: 100%|██████████| 30/30 [00:02<00:00, 13.82it/s]


0.965
[0.93, 0.924, 0.942, 0.959, 0.907, 0.919, 0.888, 0.883, 0.919, 0.912, 0.965, 0.918, 0.9, 0.941, 0.93, 0.942, 0.924, 0.865, 0.947, 0.883, 0.901, 0.93, 0.878, 0.942, 0.947, 0.93, 0.942, 0.895, 0.942, 0.93]
Updated Pheromone Levels:
 [ 9.9537  0.162   4.6098  6.5576  1.8799  6.0034  3.45    1.8451  6.9808
  0.162   6.5492  0.162   0.162   6.5608  0.162   0.162   0.162   0.162
  1.9298  1.8307 20.6916  1.9344 10.3036  7.383   5.633   1.8813 11.8104
  7.0373  5.5611  4.6368]
New ants population
: [[0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 

...

0.971
[0.913, 0.947, 0.942, 0.965, 0.924, 0.948, 0.907, 0.93, 0.942, 0.93, 0.959, 0.942, 0.912, 0.942, 0.924, 0.965, 0.918, 0.941, 0.953, 0.936, 0.924, 0.93, 0.907, 0.942, 0.953, 0.901, 0.942, 0.895, 0.971, 0.942]
Updated Pheromone Levels:
 [17.17533  1.0928   8.85882 12.65684  3.53991 12.03906  5.826    4.45059
 10.05072  0.1458  10.68928  0.1458   0.1458  10.61472  0.1458   0.1458
  0.1458   0.1458   2.68982  2.58363 35.25444  2.67096 19.25024 15.1227
  8.8817   2.59417 19.10736 11.70357  8.88899  7.94112]
New ants population
: [[0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0

Fitness Score Calculation Progress: 100%|██████████| 30/30 [00:02<00:00, 10.12it/s]


0.977
[0.913, 0.941, 0.93, 0.959, 0.942, 0.942, 0.942, 0.947, 0.918, 0.93, 0.942, 0.959, 0.924, 0.953, 0.918, 0.936, 0.942, 0.947, 0.923, 0.93, 0.918, 0.924, 0.9, 0.942, 0.953, 0.913, 0.942, 0.93, 0.959, 0.977]
Updated Pheromone Levels:
 [25.500797  1.92452  13.552938 20.022156  6.011919 18.371154  9.0114
  7.793531 12.717648  0.13122  17.156352  0.13122   0.13122  17.177248
  0.13122   0.13122   0.13122   0.13122   3.343838  3.255267 53.760996
  3.327864 28.125216 23.97243  12.75853   3.247753 28.500624 17.973213
 11.836091 11.055008]
New ants population
: [[0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0]
 [1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 

Fitness Score Calculation Progress: 100%|██████████| 30/30 [00:03<00:00,  8.97it/s]

0.971
[0.919, 0.935, 0.918, 0.941, 0.936, 0.959, 0.947, 0.953, 0.948, 0.919, 0.924, 0.953, 0.947, 0.941, 0.93, 0.942, 0.936, 0.941, 0.918, 0.924, 0.913, 0.924, 0.912, 0.93, 0.947, 0.947, 0.942, 0.953, 0.971, 0.959]
Updated Pheromone Levels:
 [36.7357173  2.667068  18.6236442 31.1939404  9.1547271 25.1650386
 11.89826   10.8261779 15.2378832  0.118098  23.7567168  0.118098
  0.118098  25.8105232  0.118098   0.118098   0.118098   0.118098
  3.9274542  3.8537403 73.0358964  3.9190776 38.9926944 35.525187
 17.164677   3.8699777 37.8965616 25.7058917 14.5364819 13.7855072]
New ants population
: [[0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0]
 [1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0]
 [1 0 1 0 0 0 1 0 0 0 0 0